## Update and install packages

In [2]:
# Update and install display packages and stable baseline 3
# uncomment if running notebook for the first time
!apt-get update && apt-get install swig cmake -y
!apt-get update && apt-get install ffmpeg freeglut3-dev xvfb -y
!pip install box2d-py
!pip install moviepy
!pip install "stable-baselines3[extra]>=2.0.0a4"
!pip install tensorboard


Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]   
Get:2 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]             
Get:3 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [4069 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.9 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [4090 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1275 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:13 http://archive.ubun

## Import Libraries

In [2]:
# Import rquired libraries and modules
import os
import signal
import subprocess
import gymnasium as gym
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import torch
from pathlib import Path
import base64
from IPython import display as ipythondisplay
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.logger import configure
import tensorboard


In [3]:
# Check gym and stable baseline 3 versions
print(f"{gym.__version__=}")
print(f"{stable_baselines3.__version__=}")


gym.__version__='0.29.1'
stable_baselines3.__version__='2.4.0a10'


## Settings

### Tunning parameters

In [4]:
# parameters for tunning the model
param_n_stack = 8                
param_clip_range = 0.07
param_learning_rate = 2.5e-4
param_batch_size = 512
param_neurons = 64
param_total_timesteps = 15000000
save_interval = param_total_timesteps/10

### Other settings

In [5]:
# name of the environment to run
env_name = 'BreakoutNoFrameskip-v4'
env_prefix = 'breakout'

# output directoies
base_dir = '.'
output_dir = os.path.join(base_dir, 'output')
env_dir =  os.path.join(output_dir, env_prefix)
logs_dir = os.path.join(env_dir, 'logs')
models_dir = os.path.join(env_dir, 'models')
videos_dir = os.path.join(env_dir, 'videos')

os.makedirs(logs_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)
os.makedirs(videos_dir, exist_ok=True)

# tensorboard name for algorithm logs
tb_log_name = 'PPO'
model_prefix = 'ppo'
model_name_final = f"{model_prefix}_model_final"

print(logs_dir)
print(models_dir)
print(videos_dir)

./output/breakout/logs
./output/breakout/models
./output/breakout/videos


## Callbacks and Directory Setup

In [6]:
# class to manage display
class Display:
    def __init__(self, command: str):
        self.command = command
        
    def start(self):
        self.process = subprocess.Popen(self.command.split())  
        os.environ['DISPLAY'] = ':1'
        
    def terminate(self):
        self.process.terminate()
        
# callback for saving model at regular intervals
class SaveOnIntervalCallback(BaseCallback):
    def __init__(self, save_interval: int, save_path: str, verbose=1):
        super().__init__(verbose)
        self.save_interval = save_interval
        self.save_path = save_path

    def _on_step(self) -> bool:
        # Save the model every 'save_interval' steps
        if self.num_timesteps % self.save_interval == 0:
            save_file = os.path.join(self.save_path, f'{model_prefix}_model_{self.num_timesteps}')
            self.model.save(save_file)
            if self.verbose > 0:
                print(f'Saving model to {save_file}.zip')
        return True


## Custom Feature Extractor

In [7]:
# Feature extraction from frames as observations / states
class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Box, features_dim: int = 512):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # convolutional layers
        self.cnn = nn.Sequential(
            nn.Conv2d(observation_space.shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, param_neurons, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        # flatten output tensor
        with torch.no_grad():
            n_flatten = self.cnn(
                torch.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        # fully conntected output layer
        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        # Process the observations through the CNN and linear layers
        return self.linear(self.cnn(observations))


## Initialise model with atari environemnt

In [8]:
# Initialize the Atari environment with the specified game and configurations
env = make_atari_env(env_name, n_envs=4, seed=0)
# Stack 4 consecutive frames together to provide temporal information
env = VecFrameStack(env, n_stack=param_n_stack) # can fine tunenumber of frame images to learn from. Always more than 1. 4 frames in each state

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [9]:
# Initialize the PPO agent with specified parameters
model = PPO(
    env=env,
    policy='CnnPolicy',
    verbose=1,
    clip_range=param_clip_range,
    ent_coef=0.01,
    learning_rate=param_learning_rate,
    n_epochs=4,
    n_steps=128,
    vf_coef=0.5,
    batch_size=param_batch_size,
    policy_kwargs={'features_extractor_class': CustomCNN},
    tensorboard_log=logs_dir
)

Using cuda device
Wrapping the env in a VecTransposeImage.


## Model training

In [14]:
display = Display("Xvfb :1 -screen 0 1024x768x24")
display.start()

(EE) 
Fatal server error:
(EE) Server is already active for display 1
	If this server is no longer running, remove /tmp/.X1-lock
	and start again.
(EE) 


In [ ]:
# Define the interval at which models are saved during training
save_callback = SaveOnIntervalCallback(save_interval, models_dir)
model.learn(total_timesteps=param_total_timesteps, progress_bar=True, callback=save_callback)

# Save the final model after training completes
final_model_path = os.path.join(models_dir, model_name_final)
model.save(final_model_path)

## Results rendering

### Video recording functions

In [10]:
# Functions to record videos of the agent playing and display the videos

def show_videos(video_path="", prefix=""):
    """Displays videos from a specified directory."""
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            '''<video alt="{0}" autoplay
                      loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{1}" type="video/mp4" />
            </video>'''.format(mp4, video_b64.decode('ascii'))
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


def record_video(env_id, model, video_length=500, prefix="", video_folder=videos_dir):
    eval_env = make_atari_env(env_id, n_envs=1, seed=0, vec_env_cls=DummyVecEnv)  # Create a single Atari environment
    eval_env = VecFrameStack(eval_env, n_stack=param_n_stack)  # Stack 4 frames together for temporal information
    eval_env = VecVideoRecorder(  # Initialize video recorder
        eval_env,
        video_folder=video_folder,  # Specify folder to save videos
        record_video_trigger=lambda step: step == 0,  # Set trigger to start recording at step 0
        video_length=video_length,  # Set video length
        name_prefix=prefix,  # Set prefix for video filenames
    )
    obs = eval_env.reset()  # Reset the environment to get initial observation
    for _ in range(video_length):  # Loop through for the specified video length
        action, _ = model.predict(obs)  # Predict action based on current observation
        obs, _, _, _ = eval_env.step(action)  # Execute action in the environment
    eval_env.close()  # Close the environment and video recorder


In [11]:
# Display the video
import os

def get_model_identifiers(models_dir):
    files = os.listdir(models_dir)
    model_files = [f for f in files if f.startswith(f'{model_prefix}_model_')]
    identifiers = [f.split('_')[2] for f in model_files]
    return identifiers

def find_key_identifiers(identifiers):
    identifiers.sort()  # Ensure identifiers are sorted
    earliest = identifiers[0]
    final = identifiers[-1]
    middle = identifiers[len(identifiers) // 2]
    return earliest, middle, final

def view(models_dir, video_length=1000):
    print("MODEL DIR", models_dir)
    identifiers = get_model_identifiers(models_dir)
    earliest, middle, final = find_key_identifiers(identifiers)

    # Record videos at the beginning, middle, and end of training
    for stage, identifier in zip(["beginning", "middle", "end"], [earliest, middle, final]):
        model_path = os.path.join(models_dir, f'{model_prefix}_model_{identifier}')
        print("MODEL PATH", model_path)
        model = PPO.load(model_path)
        record_video(env_name, model, video_length=video_length, prefix=f'{model_prefix}-{env_prefix}-{stage}')

    # Display the videos
    for stage in ["beginning", "middle", "end"]:
        show_videos("videos", prefix=f'{model_prefix}-{env_prefix}-{stage}')



### Recording videos

In [12]:
#record video using background display
display = Display("Xvfb :1 -screen 0 1024x768x24")
display.start()

The XKEYBOARD keymap compiler (xkbcomp) reports:
> Internal error:   Could not resolve keysym XF86AudioPreset
> Internal error:   Could not resolve keysym XF86MonBrightnessCycle
> Internal error:   Could not resolve keysym XF86WWAN
> Internal error:   Could not resolve keysym XF86RFKill
> Internal error:   Could not resolve keysym XF86Keyboard
> Internal error:   Could not resolve keysym XF86RotationLockToggle
> Internal error:   Could not resolve keysym XF86FullScreen
Errors from xkbcomp are not fatal to the X server


In [13]:
view(models_dir, video_length=3000)

MODEL DIR ./output/breakout/models
MODEL PATH ./output/breakout/models/ppo_model_10500000.zip


/opt/conda/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Saving video to /root/RL/A2/output/breakout/videos/ppo-breakout-beginning-step-0-to-step-3000.mp4
Moviepy - Building video /root/RL/A2/output/breakout/videos/ppo-breakout-beginning-step-0-to-step-3000.mp4.
Moviepy - Writing video /root/RL/A2/output/breakout/videos/ppo-breakout-beginning-step-0-to-step-3000.mp4



Moviepy - Done !
Moviepy - video ready /root/RL/A2/output/breakout/videos/ppo-breakout-beginning-step-0-to-step-3000.mp4
MODEL PATH ./output/breakout/models/ppo_model_3000000.zip


/opt/conda/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Saving video to /root/RL/A2/output/breakout/videos/ppo-breakout-middle-step-0-to-step-3000.mp4
Moviepy - Building video /root/RL/A2/output/breakout/videos/ppo-breakout-middle-step-0-to-step-3000.mp4.
Moviepy - Writing video /root/RL/A2/output/breakout/videos/ppo-breakout-middle-step-0-to-step-3000.mp4



Moviepy - Done !
Moviepy - video ready /root/RL/A2/output/breakout/videos/ppo-breakout-middle-step-0-to-step-3000.mp4
MODEL PATH ./output/breakout/models/ppo_model_final.zip
Saving video to /root/RL/A2/output/breakout/videos/ppo-breakout-end-step-0-to-step-3000.mp4
Moviepy - Building video /root/RL/A2/output/breakout/videos/ppo-breakout-end-step-0-to-step-3000.mp4.
Moviepy - Writing video /root/RL/A2/output/breakout/videos/ppo-breakout-end-step-0-to-step-3000.mp4



Moviepy - Done !
Moviepy - video ready /root/RL/A2/output/breakout/videos/ppo-breakout-end-step-0-to-step-3000.mp4


In [ ]:
display.terminate()

In [ ]:
# !tensorboard --logdir {logs_dir} --host 0.0.0.0 --port 6006

In [ ]:
# # Function for plotting the learning curve of the agent
# def plot_results(log_folder, title="Learning Curve"):
#     x, y = ts2xy(load_results(log_folder), "timesteps")
#     y = np.convolve(y, np.ones((50,))/50, mode='valid')
#     x = x[len(x) - len(y):]
#     plt.figure(figsize=(10,5))
#     plt.plot(x, y)
#     plt.xlabel("Number of Timesteps")
#     plt.ylabel("Rewards")
#     plt.title(title + " Smoothed")
#     plt.show()

# plot_results(logs_dir)
